In [1]:
from hepmc import *
import numpy as np

In [2]:
sarge = densities.Sarge(2, 3, 100.)

In [3]:
eeqqg = densities.ee_qq_ng(1, 100., 5., .3)

In [4]:
rambo_mapping = phase_space.RamboOnDiet(100., 3)
mapped = phase_space.MappedDensity(eeqqg, rambo_mapping)

In [12]:
%time sarge_sample = ImportanceMC(sarge)(eeqqg.pdf, 10000)

Event 1	(avg. trials per event: 7.000000)
Event 2	(avg. trials per event: 9.000000)	XS = 53.387515 pb +- ( 52.416450 pb = 98.181099 %)
Event 3	(avg. trials per event: 6.333333)	XS = 64.370174 pb +- ( 44.514934 pb = 69.154596 %)
Event 4	(avg. trials per event: 6.250000)	XS = 136.264755 pb +- ( 77.868765 pb = 57.145198 %)
Event 5	(avg. trials per event: 5.200000)	XS = 152.033823 pb +- ( 73.441638 pb = 48.306118 %)
Event 6	(avg. trials per event: 4.666667)	XS = 222.843506 pb +- ( 85.556269 pb = 38.392983 %)
Event 7	(avg. trials per event: 4.714286)	XS = 220.477589 pb +- ( 71.577951 pb = 32.464955 %)
Event 8	(avg. trials per event: 4.375000)	XS = 210.845019 pb +- ( 71.945723 pb = 34.122562 %)
Event 9	(avg. trials per event: 4.000000)	XS = 213.594703 pb +- ( 71.454514 pb = 33.453317 %)
Event 10	(avg. trials per event: 4.600000)	XS = 167.335788 pb +- ( 58.541066 pb = 34.984188 %)
Event 20	(avg. trials per event: 4.500000)	XS = 166.022807 pb +- ( 39.758966 pb = 23.947894 %)
Event 30	(avg. tri

In [13]:
print('estimated ESS:', util.effective_sample_size(sarge_sample, sarge_sample.mean, sarge_sample.variance))

estimated ESS: [10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000.
 10000. 10000.]


In [ ]:
densities.export_hepmc(100., sarge_sample, "../samples/qcd/2-3/sarge_weighted.hepmc")

In [ ]:
bound = sarge_sample.weights.max()

In [ ]:
%%time
# acceptance-rejection sampler
sarge_sample_unweighted = np.empty((10000, 12))
for i in range(10000):
    aprob = 0
    while np.random.random() > aprob:
        proposal = sarge.proposal()
        f = eeqqg.pdf(proposal)
        g = sarge.proposal_pdf(None, proposal)
        aprob = f/(bound*g)
        if aprob > 1:
            print("bound too low")
    sarge_sample_unweighted[i] = proposal
sarge_sample_unweighted = Sample(data=sarge_sample_unweighted, weights=np.full(10000, 1./10000))

In [ ]:
densities.export_hepmc(100., sarge_sample_unweighted, "../samples/qcd/2-3/sarge_unweighted.hepmc")

In [ ]:
rambo_sample = PlainMC(5)(mapped, 10000)
rambo_sample.integral, rambo_sample.integral_err

In [ ]:
ys = mapped.pdf(rambo_sample.data)
max_y = ys.max()

In [ ]:
rambo = AcceptRejectSampler(mapped, max_y, 5)
%time rambo_sample = rambo.sample(1000)

In [ ]:
rambo_sample.data = rambo_mapping.map(rambo_sample.data)
rambo_sample.weights = np.full(rambo_sample.size, 1./rambo_sample.size)

In [ ]:
densities.export_hepmc(100., rambo_sample, "../samples/qcd/2-3/rambo.hepmc")

In [15]:
# Metropolis with Rambo mapping
met = DefaultMetropolis(5, mapped, proposals.Gaussian(5, .01))
start = met.sample(5000, np.random.rand(5)).data[-1] # warmup
%time met_sample = met.sample(10000, start)

Event 1	(batch acceptance rate: 0.000000)
Event 2	(batch acceptance rate: 0.000000)	max sequence length: 3
Event 3	(batch acceptance rate: 0.333333)	max sequence length: 3
Event 4	(batch acceptance rate: 0.250000)	max sequence length: 3
Event 5	(batch acceptance rate: 0.400000)	max sequence length: 3
Event 6	(batch acceptance rate: 0.500000)	max sequence length: 3
Event 7	(batch acceptance rate: 0.428571)	max sequence length: 3
Event 8	(batch acceptance rate: 0.375000)	max sequence length: 3
Event 9	(batch acceptance rate: 0.333333)	max sequence length: 4
Event 10	(batch acceptance rate: 0.300000)	max sequence length: 5
Event 20	(batch acceptance rate: 0.200000)	max sequence length: 10
Event 30	(batch acceptance rate: 0.133333)	max sequence length: 15
Event 40	(batch acceptance rate: 0.125000)	max sequence length: 17
Event 50	(batch acceptance rate: 0.220000)	max sequence length: 17
Event 60	(batch acceptance rate: 0.200000)	max sequence length: 17
Event 70	(batch acceptance rate: 0.27

In [16]:
print('estimated ESS:', util.effective_sample_size(met_sample, met_sample.mean, met_sample.variance))

estimated ESS: [116.51390124  26.54519491  57.09120015  40.30516534  30.44438605]


In [ ]:
met_sample.data = rambo_mapping.map(met_sample.data)
met_sample.weights = np.full(met_sample.size, 1./met_sample.size)
densities.export_hepmc(100., met_sample, "../samples/qcd/2-3/metropolis.hepmc")

In [37]:
# adaptive Metropolis
def adapt_schedule(t):
    if t < 50000:
        return True
    else:
        return False
    
met = AdaptiveMetropolisUpdate(5, mapped, proposals.Gaussian(5, .01), t_initial=10000, adapt_schedule=adapt_schedule)
%time met_sample = met.sample(1060000, np.random.rand(5))
met_sample.data = met_sample.data[60000:] # discard warmup samples

Event 1	(batch acceptance rate: 1.000000)
Event 2	(batch acceptance rate: 0.500000)	max sequence length: 2
Event 3	(batch acceptance rate: 0.666667)	max sequence length: 2
Event 4	(batch acceptance rate: 0.500000)	max sequence length: 2
Event 5	(batch acceptance rate: 0.400000)	max sequence length: 3
Event 6	(batch acceptance rate: 0.333333)	max sequence length: 4
Event 7	(batch acceptance rate: 0.285714)	max sequence length: 5
Event 8	(batch acceptance rate: 0.375000)	max sequence length: 5
Event 9	(batch acceptance rate: 0.444444)	max sequence length: 5
Event 10	(batch acceptance rate: 0.400000)	max sequence length: 5
Event 20	(batch acceptance rate: 0.300000)	max sequence length: 7
Event 30	(batch acceptance rate: 0.366667)	max sequence length: 7
Event 40	(batch acceptance rate: 0.275000)	max sequence length: 14
Event 50	(batch acceptance rate: 0.240000)	max sequence length: 15
Event 60	(batch acceptance rate: 0.216667)	max sequence length: 15
Event 70	(batch acceptance rate: 0.2142

In [38]:
# thin sample
met_sample.data = met_sample.data[::100]

In [39]:
met_sample.data = rambo_mapping.map(met_sample.data)
met_sample.weights = np.full(met_sample.size, 1./met_sample.size)
densities.export_hepmc(100., met_sample, "../samples/qcd/2-3/metropolis.hepmc")

In [29]:
print('estimated ESS:', util.effective_sample_size(met_sample, met_sample.mean, met_sample.variance))

estimated ESS: [74.73137319 75.05706785 76.53092459 93.0561817  67.43079625]


In [33]:
# dual averaging Metropolis
def adapt_schedule(t):
    if t < 5000:
        return True
    else:
        return False
    
met = StochasticOptimizeUpdate(mapped, proposals.Gaussian(5, .01), adapt_schedule=adapt_schedule)
%time met_sample = met.sample(16000, np.random.rand(5))
met_sample.data = met_sample.data[6000:] # discard warmup samples

Event 1	(batch acceptance rate: 1.000000)
Event 2	(batch acceptance rate: 1.000000)	max sequence length: 1
Event 3	(batch acceptance rate: 0.666667)	max sequence length: 2
Event 4	(batch acceptance rate: 0.500000)	max sequence length: 3
Event 5	(batch acceptance rate: 0.400000)	max sequence length: 4
Event 6	(batch acceptance rate: 0.333333)	max sequence length: 5
Event 7	(batch acceptance rate: 0.285714)	max sequence length: 6
Event 8	(batch acceptance rate: 0.250000)	max sequence length: 7
Event 9	(batch acceptance rate: 0.222222)	max sequence length: 8
Event 10	(batch acceptance rate: 0.200000)	max sequence length: 9
Event 20	(batch acceptance rate: 0.100000)	max sequence length: 19
Event 30	(batch acceptance rate: 0.066667)	max sequence length: 29
Event 40	(batch acceptance rate: 0.050000)	max sequence length: 39
Event 50	(batch acceptance rate: 0.060000)	max sequence length: 42
Event 60	(batch acceptance rate: 0.050000)	max sequence length: 42
Event 70	(batch acceptance rate: 0.04

In [34]:
print('estimated ESS:', util.effective_sample_size(met_sample, met_sample.mean, met_sample.variance))

estimated ESS: [48.03335895 31.05819789 38.86241072 50.05685336 46.85155107]


In [17]:
# now sarge + local in MC3
importance = DefaultMetropolis(3 * 4, eeqqg, proposal=sarge)  # using sarge, full momentum sample space
local = DefaultMetropolis(5, mapped, proposals.Gaussian(5, .01))  # using rambo, rambo sample space
update = MixingMarkovUpdate(5, [importance, local], target=mapped,
                            out_maps={0: rambo_mapping.map_inverse},
                            in_maps={0: rambo_mapping.map}) # rambo sample space, important to set target explicitly
start = update.sample(5000, np.random.rand(5)).data[-1] # warmup
%time mc3_sample = update.sample(10000, start)

Event 1	(batch acceptance rate: 1.000000)
Event 2	(batch acceptance rate: 1.000000)	max sequence length: 1
Event 3	(batch acceptance rate: 0.666667)	max sequence length: 2
Event 4	(batch acceptance rate: 0.750000)	max sequence length: 2
Event 5	(batch acceptance rate: 0.600000)	max sequence length: 2
Event 6	(batch acceptance rate: 0.666667)	max sequence length: 2
Event 7	(batch acceptance rate: 0.714286)	max sequence length: 2
Event 8	(batch acceptance rate: 0.750000)	max sequence length: 2
Event 9	(batch acceptance rate: 0.777778)	max sequence length: 2
Event 10	(batch acceptance rate: 0.800000)	max sequence length: 2
Event 20	(batch acceptance rate: 0.650000)	max sequence length: 4
Event 30	(batch acceptance rate: 0.500000)	max sequence length: 10
Event 40	(batch acceptance rate: 0.425000)	max sequence length: 10
Event 50	(batch acceptance rate: 0.420000)	max sequence length: 10
Event 60	(batch acceptance rate: 0.483333)	max sequence length: 10
Event 70	(batch acceptance rate: 0.528

In [19]:
print('estimated ESS:', util.effective_sample_size(mc3_sample, mc3_sample.mean, mc3_sample.variance))

estimated ESS: [341.60612756 276.66878284 240.30061058 213.43745973 228.29383859]


In [ ]:
mc3_sample.data = rambo_mapping.map(mc3_sample.data)
mc3_sample.weights = np.full(mc3_sample.size, 1./mc3_sample.size)
densities.export_hepmc(100., mc3_sample, "../samples/qcd/2-3/mc3.hepmc")